In [1]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import nibabel as nib
import numpy as np
import os
from skimage import data, util, measure, morphology
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity
from sklearn import preprocessing as pre
import pandas as pd


#WORK_DIR="/home/storm/PhD_open_dataset/open_ms_data/cross_sectional/coregistered_resampled/"

WORK_DIR="/users/kfotso/carc-scratch/openms_dataset/coregistered_resampled"


# Defining the total bins and the hist feature array
total_bins = 10
features = np.zeros([1, total_bins])
PATIENTS = sorted(os.listdir(WORK_DIR))
IND = 0 

# Create dictionary to retain coordinates of lesions:
total_patients = 30
patient_dict = dict(zip(PATIENTS, [{"indexes" : [], 
                                    "cluster_types" : [],
                                    "total_cluster_types" : [] } for _ in range(total_patients) ]))

for patient_num, DIR in enumerate(PATIENTS):
    
    print("####### Working on patient number {} #########".format(patient_num ))
    patient = os.path.join(WORK_DIR, DIR)

    # Loading the FLAIR dataset
    flair_path = os.path.join(patient, "FLAIR_advanced_gabor_amfm_high.nii.gz")
    lesion_path = os.path.join(patient, "consensus_gt.nii.gz")
    
    flair_nii = nib.load(flair_path)
    gt_nii = nib.load(lesion_path)
    
    # Transforming data to numpy array
    flair_arr = flair_nii.get_fdata()
    
    # converting nan to num
    flair_arr = np.nan_to_num(flair_arr)
    
    gt_arr = gt_nii.get_fdata().astype(bool)
    
    # Normalize the flair image:
    flair_arr = (flair_arr-np.min(flair_arr))/(np.max(flair_arr)-np.min(flair_arr))
    
    # Preprocessing step:
    #We clean the lesion volume of a very small lesions
    thresh_mask = morphology.remove_small_objects(gt_arr, min_size=15) 
    label_image = measure.label(thresh_mask, connectivity=gt_arr.ndim)
       
    #Connected components
    props = measure.regionprops(label_image)
    properties = ['label', 'area', 'centroid']
    props_table = measure.regionprops_table(label_image,
                           properties=properties)
    
    data = pd.DataFrame(props_table)


    # Get only the lesions based on the ground truth.
    # lesion_only = flair_arr * gt_arr
    
    #
    sl, x, y = flair_arr.shape
    
    #features = np.zeros([props_table['label'].shape[0], total_bins])
    
    if patient_num == 0:
        patient_dict[DIR]["indexes"].append(IND)
    else:
        patient_dict[DIR]["indexes"].append(IND + 1)
        
    for ind in np.nditer(props_table['label']): 
        region = label_image==ind
        lesion_only = flair_arr * region
        
        
        # histogram
        les = np.where(lesion_only > 0)
        h, edges = np.histogram(lesion_only[les], bins=total_bins)
        
        # Applying kernel density
        #h = h[:, np.newaxis]
        #kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(h)
        #h_kde = kde.score_samples(kde)
        
        if patient_num == 0 and ind ==0:
            features[0, :] = h
        else:        
            features = np.vstack((features, h))
            #features[ind -1] = h
            
            
    #
    IND += np.ndarray.item(ind)
    patient_dict[DIR]["indexes"].append(IND)

####### Working on patient number 0 #########
####### Working on patient number 1 #########
####### Working on patient number 2 #########
####### Working on patient number 3 #########
####### Working on patient number 4 #########
####### Working on patient number 5 #########
####### Working on patient number 6 #########
####### Working on patient number 7 #########
####### Working on patient number 8 #########
####### Working on patient number 9 #########
####### Working on patient number 10 #########
####### Working on patient number 11 #########
####### Working on patient number 12 #########
####### Working on patient number 13 #########
####### Working on patient number 14 #########
####### Working on patient number 15 #########
####### Working on patient number 16 #########
####### Working on patient number 17 #########
####### Working on patient number 18 #########
####### Working on patient number 19 #########
####### Working on patient number 20 #########
####### Working on pati

In [2]:
# Clustering for all the patients    
total_clusters = 2
clustering = KMeans(n_clusters=total_clusters,
                    random_state=0, n_init="auto",).fit(features)

# Print Clustering #
print("Total clusters are {}".format(clustering.n_clusters))

for patient_ in patient_dict.keys():
    start_ind = patient_dict[patient_]["indexes"][0]
    end_ind = patient_dict[patient_]["indexes"][1]
    
    cluster_types = np.unique(clustering.labels_[start_ind:end_ind])
    patient_dict[patient_]["cluster_types"] = cluster_types.tolist()
    patient_dict[patient_]["total_cluster_types"] = len(cluster_types.tolist())

# Converting into a dataframe and saving as csv:
SOFTWARE_DIR = "/users/kfotso/carc-scratch/openms_dataset/results"
pd_clusters = pd.DataFrame.from_dict(patient_dict)
pd_clusters.to_csv(os.path.join(SOFTWARE_DIR, "_knn_cluster_types_mag_high_2_clusters.csv"))

Total clusters are 2
